# Define necessary functions: 

#### Get the incidence matrix N of G:

0: the 1 snark on 10 vertices, 
1, 2: the 2 snarks on 18 vertices,
3 - 8: the 6 snarks on 20 vertices,
9 - 39: the 31 snarks on 22 vertices, 
40 - 194: the 155 snarks on 24 vertices;

In [ ]:
def incidence_matrix (i):
    
    # Import snarks from a .txt file based on House of Snarks, https://hog.grinvin.org/Snarks :
    with open('/Users/zuzapalion/Desktop/Snarks.txt') as f:
        snarks = [line.rstrip() for line in f]
    
    # Encode graph6 notation from snark[i] to a graph G: 
    from sage.graphs.graph_input import from_graph6
    G = Graph()
    from_graph6(G, snarks[i]) 
    
    # Obtain the incidence incidence matrix N of the graph G over GF(2):   
    N = G.incidence_matrix()
    N = N.change_ring(GF(2))
    return N 

#### Get the number of vertices vxs of G:

In [ ]:
def vertices(N):
    vxs = N.dimensions()[0]
    return vxs

#### Define an ordering for snarks on different numbers of vertices:

In [ ]:
def count(vxs, i):
    no = 0
    if vxs == 10:
        no = i
    if vxs == 18:
        no = i-1
    if vxs == 20:
        no = i-3
    if vxs == 22:
        no = i-9
    if vxs == 24:
        no = i-40
    no = no + 1
    return no

#### Get a matrix M_t with rows that are all of T-joins of G:

In [ ]:
def get_all_tjoins_matrix(N):
    # Obtain the kernel K of the incidence matrix of the graph G:
    K = N.right_kernel_matrix(basis='computed') 

    # Create a list B of basis vectors (circuits of G) for the kernel K:
    B = K.rows()

    # Obtain all the cycles of graph G, and matrix S with cycles as columns:
    V = VectorSpace(GF(2),N.dimensions()[1])
    cycles = V.subspace(B).list()
    S = matrix(GF(2), cycles)

    # Find a single perfect matching for T = V(G) by solving the feasibility problem: 
    R = N.change_ring(QQ)
    p_t = MixedIntegerLinearProgram(solver = "GLPK")
    x_t = p_t.new_variable(binary = True)
    p_t.add_constraint(R * x_t == 1)
    p_t.set_objective(0)
    p_t.solve()
    x_t = p_t.get_values(x_t) 
    x_t_v = vector(GF(2), x_t)
    b = N*x_t_v
    
    one = []
    for i in range(N.dimensions()[0]):
        one.append(1)
    b == one
    
    # Create a matrix Z whose every column is the single perfect matching:   
    rows = []
    for i in range(S.dimensions()[0]):
        rows.append(x_t_v)
    
    Z = matrix(GF(2), rows)
    
    # Obtain a matrix M_t with rows being all of T-joins of Snark i: 
    M = S+Z
    M_t = M.transpose()
    
    return M_t

#### Solve the dual LP to find the incidence vector "packing" of the maximum fractional packing of T-joins of G: 

In [ ]:
def get_tjoins_packing(M_t):
    
    # Solve the dual LP: find the max fractional packing of T-joins 
    d = MixedIntegerLinearProgram(solver = "GLPK/exact", maximization = True)
    y = d.new_variable(real=True, nonnegative=True)
    d.add_constraint(M_t * y <= 1)
    d.set_objective(d.sum(y[i] for i in range(M_t.dimensions()[1])))
    sol = d.solve()
    packing = d.get_values(y)
    
    return packing 

#### Show only non-zero entries of "packing" in "packing_short":

In [ ]:
def shorten_packing(packing):
    
    # Show only the non-zero entries of the incidence vector of the packing: 
    packing_short = {}  
    for key, value in packing.items():
         if (value != 0):
                packing_short[key] = value
    
    return packing_short

#### Check if n is a power of 2: 

In [ ]:
def power_of_two(n):
    while (n != 1):
            if (n % 2 != 0):
                return False
            else:
                n = n // 2       
    
    return True

#### Check if the entry of "packing_short" is dyadic:

In [ ]:
def check_dyadicness_entry(packing_short):
    
    # Check if each entry of the incidence vector of the packing is dyadic:
    packing_short_m = {}
    
    i = 1 # i is the integer, that increments in the while loop, which if multiplied with the value gives an integer
    j = 0 # j is the integer that tracks the number of dyadic entries in the incidence vector

    for key in packing_short:
        rounded = n(packing_short[key], prec = 32)
        while i >= 1:
            temp = rounded*i
            if temp.is_integer()==True:
                packing_short_m[key] = [packing_short[key], i, temp] #store for reference
                break
            else:
                i=i+1
  
        if(power_of_two(i)):
            j=j+1
            
    return j 

#### Check if the incidence vector "packing_short" is dyadic:

In [ ]:
def check_dyadicness_vector(packing_short, j):
    if (j==len(list(packing_short.values()))):
    
        return True 

# Investigate the snarks: 

In [ ]:
# Record the time for the code to run:
import time
start_time = time.time()
meas = [] # to measure the total time 

dyadic = 0 # to count the total number of dyadic optimal solutions


# Iterate through 194 snarks to look for maximum fractional packings of postman sets that are dyadic:
for i in range(195):
    N = incidence_matrix(i)
    M_t = get_all_tjoins_matrix(N)
    packing = get_tjoins_packing(M_t)
    short_packing = shorten_packing(packing)
    
    check = check_dyadicness_entry(short_packing)
    final = check_dyadicness_vector(short_packing, check)
    
    vxs = vertices(N)
    no = count(vxs, i)
    
    passed = (time.time() - start_time)
    howlong = round(passed, 2)
    meas.append(passed)  
    start_time = time.time()
    
    if final == True:
        print("The incidence vector of Snark #",no,"with", vxs, "vertices:", short_packing, " is dyadic.")
        print("The operation took", howlong, "seconds. \n")
        dyadic = dyadic + 1 

    else:
        print("The incidence vector of Snark #",no,"with", vxs, "vertices:", short_packing, " is not dyadic.")
        print("The operation took", howlong, "seconds. \n")


# Print the % of snarks giving dyadic optimal solutions, and the total run time of the program:        
print(n(float(dyadic/194), digits = 2)*100, "% of snarks give a dyadic optimal solution to the dual LP of finding the maximum fractional packing of postam sets.")
print("\nTotal run time:", round(sum(meas)/60, 2), "minutes")